In [1]:
import pandas as pd
import os
import sys
# Get the current directory
current_dir = os.getcwd()
# Move one directory up
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
# Add the parent directory to the system path
sys.path.insert(0, parent_dir)
from OTAs.file_management.file_path_manager import FilePathManager




In [4]:
df = pd.read_excel(r"C:\Users\Wojciech\Documents\Python\MyOTAs\OTAs_Application\MyOTAs\working_files\Rimini 2024 TTG EXPO Italy.xlsx")
df = pd.read_excel(r"C:\Users\Wojciech\Documents\Python\MyOTAs\OTAs_Application\MyOTAs\working_files\WTM 2024.xlsx")
df

,Operator
0,1StopBorneo Wildlife
1,2Xceed
2,360 Tours Lanka
3,365 TRAVEL
4,4cycling 'n trek
...,...
1436,Saladino Tours Egypt
1437,Stena Line
1438,Think Bali Holidays
1439,TRAVEL BEFORE


In [5]:
df_operators = pd.DataFrame()
sites = ['GYG', 'Viator', 'Musement', 'Headout']
for site in sites:
    file_manager = FilePathManager(site, 'N/A')
    file_path_xlsx_operator = file_manager.get_file_paths()['file_path_xlsx_operator']
    print(file_path_xlsx_operator)
    temp_df = pd.read_excel(file_path_xlsx_operator)
    temp_df['Site'] = file_path_xlsx_operator.split('_')[-1].split('.')[0]
    df_operators = pd.concat([df_operators, temp_df])
    


G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG.xlsx
G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Viator.xlsx
G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Musement.xlsx
G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Headout.xlsx


In [6]:
import pandas as pd

# Assume df_operators is your DataFrame

# Step 1: Exclude unwanted entries and convert 'Reviews' to numeric
df_operators = df_operators[
    (df_operators['Operator'] != 'ToDo') & 
    (df_operators['Operator'] != 'InccorectURL')
]
df_operators['Reviews'] = pd.to_numeric(df_operators['Reviews'], errors='coerce')

# Step 2: Compute basic operator statistics
operator_stats = df_operators.groupby('Operator').agg(
    activity_count=('uid', 'nunique'),
    max_reviews=('Reviews', 'max'),
    min_reviews=('Reviews', 'min'),
    avg_reviews=('Reviews', 'mean')
).reset_index()

# Step 3: Calculate counts per 'Operator' and 'Site'
operator_site_counts = df_operators.groupby(['Operator', 'Site'])['uid'].nunique().reset_index()

# Step 4: Pivot the 'Site' column
operator_site_pivot = operator_site_counts.pivot(
    index='Operator', 
    columns='Site', 
    values='uid'
).reset_index()

# Step 4a: Fill NaN values with 0
operator_site_pivot.fillna(0, inplace=True)

# Step 4b: Ensure the site columns are integers
sites = ['GYG', 'Viator', 'Musement']
for site in sites:
    if site in operator_site_pivot.columns:
        operator_site_pivot[site] = operator_site_pivot[site].astype(int)
    else:
        # If the site is missing, add it with zeros
        operator_site_pivot[site] = 0

# Step 5: Merge with operator_stats
operator_stats = operator_stats.merge(
    operator_site_pivot[['Operator'] + sites], 
    on='Operator', 
    how='left'
)

# Step 5a: Fill any remaining NaN values with 0
operator_stats[sites] = operator_stats[sites].fillna(0).astype(int)

# Display the final DataFrame
operator_stats


,Operator,activity_count,max_reviews,min_reviews,avg_reviews,GYG,Viator,Musement
0,0,2,29.0,0.0,14.50,0,2,0
1,2,4,1.0,0.0,0.25,0,4,0
2,7,1,0.0,0.0,0.00,0,1,0
3,1388,1,0.0,0.0,0.00,0,1,0
4,1515,1,3.0,3.0,3.00,1,0,0
...,...,...,...,...,...,...,...,...
33216,ΒΑΣΙΛΟΠΟΥΛΟΥ Μ. -ΤΣΙΑΤΣΙΟΣ Ν. Ο.Ε,1,1.0,1.0,1.00,0,0,1
33217,Мадрид в,1,NaN,NaN,NaN,0,1,0
33218,オランダ バル・ツアー,2,NaN,NaN,NaN,2,0,0
33219,格列柯,1,NaN,NaN,NaN,1,0,0


In [7]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Clean operator names
def clean_operator_name(name):
    suffixes = [
        'ltd', 'llc', 'inc', 'srl', 'kft', 'gmbh', 'société',
        'company', 'co', 'limited', 'plc', 'bv', 'ag'
    ]
    name = name.lower()
    for suffix in suffixes:
        name = re.sub(r'\b{}\b'.format(suffix), '', name)
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

df['Operator'] = df['Operator'].astype(str)
df['clean_operator'] = df['Operator'].apply(clean_operator_name)

operator_stats['Operator'] = operator_stats['Operator'].astype(str)
operator_stats['clean_operator'] = operator_stats['Operator'].apply(clean_operator_name)

operator_stats_names = operator_stats['clean_operator'].unique()

TOP_N = 2

def get_top_matches(name, choices, scorer=fuzz.token_sort_ratio, limit=TOP_N):
    matches = process.extract(name, choices, scorer=scorer, limit=limit)
    return matches

df['matches'] = df['clean_operator'].apply(
    lambda x: get_top_matches(x, operator_stats_names)
)

df_exploded = df.explode('matches')
df_exploded[['matched_operator', 'similarity_score', '_']] = pd.DataFrame(
    df_exploded['matches'].tolist(), index=df_exploded.index
)
df_exploded = df_exploded.drop(columns=['matches', '_'])

merged_df = pd.merge(
    df_exploded,
    operator_stats,
    how='left',
    left_on='matched_operator',
    right_on='clean_operator',
    suffixes=('_df', '_stats')
)

final_df = merged_df[[
    'Operator_df',
    'matched_operator',
    'similarity_score',
    'activity_count',
    'max_reviews',
    'min_reviews',
    'avg_reviews',
    'GYG',
    'Viator',
    'Musement'
]].rename(columns={
    'Operator_df': 'Original_Operator',
    'similarity_score': 'Similarity_Score'
})

final_df = final_df.sort_values(
    by=['Original_Operator', 'Similarity_Score'], 
    ascending=[True, False]
)



In [8]:
final_df['Similarity_Score'] = final_df['Similarity_Score']/100
final_df['Similarity_Score'] = final_df['Similarity_Score'].replace('.', ',')
final_df

,Original_Operator,matched_operator,Similarity_Score,activity_count,max_reviews,min_reviews,avg_reviews,GYG,Viator,Musement
3246,#GoEurope DMC,vizit europe dmc,0.714286,2,0.0,0.0,0.000000,0,2,0
3247,#GoEurope DMC,travel dmc group,0.642857,1,0.0,0.0,0.000000,0,1,0
0,1StopBorneo Wildlife,wildlife spain,0.705882,1,NaN,NaN,NaN,1,0,0
1,1StopBorneo Wildlife,ark wildlife park,0.594595,3,35.0,3.0,18.333333,0,3,0
2,2Xceed,cedok,0.545455,12,825.0,42.0,290.333333,0,12,0
...,...,...,...,...,...,...,...,...,...,...
1966,myGO,woyago,0.600000,11,18.0,0.0,5.083333,7,4,0
2825,tuktukrental.com (Tuktuk Rental Sri Lanka),torna a surriento boat rental,0.558824,4,2.0,2.0,2.000000,4,0,0
2826,tuktukrental.com (Tuktuk Rental Sri Lanka),bike rental central park,0.539683,4,336.0,0.0,159.000000,0,4,0
3080,​​​​​​​STARLING FLEET,storytelling florence,0.685714,2,123.0,20.0,71.500000,0,2,0


In [9]:
final_df.to_excel('OuputMatch.xlsx')